In [47]:
!pip install plotly
!pip install rdflib
!pip install networkx
!pip install pyvis
!pip install matplotlib
!pip install dash-bootstrap-components
!pip install dash plotly
!pip uninstall pyvis
!pip install pyvis



^C
^C


In [49]:
# Bibliothèque pour charger des fichiers RDF en Python
from rdflib import Graph

# Bibliothèque pour manipuler des graphes
import networkx as nx

# Bibliothèque pour visualiser des graphes de manière interactive sur une page web
from pyvis.network import Network

# Bibliothèques pour créer des tableaux de bord et visualiser des données
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.graph_objs as go

# Bibliothèque pour manipuler des données sous forme de DataFrame
import pandas as pd

# Facultatif : Pour des visualisations de graphes statiques
import matplotlib.pyplot as plt


In [86]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import subprocess
import os
from rdflib import Graph

# Charger les données RDF à partir du fichier .ttl
rdf_file_path = "teaching_akg.ttl"  # Remplacez par le chemin de votre fichier
g = Graph()
g.parse(rdf_file_path, format="turtle")

# Créer l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'Interface Dash pour Projet'
server = app.server  # Pour déploiement avec des services externes
FILE_PATH = 'projet_tpe2_1.py'

# Stocker les informations d'authentification simples (pour démonstration uniquement)
USERNAME = 'utilisateur'
PASSWORD = 'motdepasse'

# Définir le layout de l'application
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Layout de la page de connexion
login_layout = html.Div([
    html.H2("Page de Connexion"),
    dbc.Input(id='username', placeholder="Nom d'utilisateur", type='text', style={'margin-bottom': '10px'}),
    dbc.Input(id='password', placeholder='Mot de passe', type='password', style={'margin-bottom': '10px'}),
    dbc.Button('Se connecter', id='login-button', n_clicks=0, color='primary'),
    html.Div(id='login-message', style={'margin-top': '20px', 'color': 'red'})
])

# Layout de la page principale après connexion
main_layout = html.Div([
    html.H1("Interface Dash pour Étude des Communautés"),
    html.Div([
        dbc.Button("Visualiser toutes les communautés séparées", id='view-all-button', color='primary', style={'margin-bottom': '10px'}),
        dbc.Input(id='activity-name', placeholder="Entrez le nom de l'activité", type='text', style={'margin-bottom': '10px'}),
        dbc.Button("Visualiser une seule communauté", id='view-single-button', color='secondary', style={'margin-bottom': '10px'}),
        dbc.Input(id='activity-names', placeholder='Entrez deux activités ou plus, séparées par des virgules', type='text', style={'margin-bottom': '10px'}),
        dbc.Button("Étudier plusieurs activités", id='view-multiple-button', color='info', style={'margin-bottom': '10px'}),
        dbc.Button("Détecter les communautés basées sur des ressources spécifiques", id='view-specific-button', color='success', style={'margin-bottom': '10px'}),
        html.Div(id='file-upload-container', children=[
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Glissez et déposez ou ',
                    html.A('sélectionnez un fichier')
                ]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed',
                    'borderRadius': '5px', 'textAlign': 'center',
                    'margin-bottom': '10px'
                },
                multiple=False
            )
        ])
    ]),
    html.Div(id='output-container', style={'margin-top': '20px'})
])

# Callback pour la redirection après connexion
@app.callback(
    [Output('url', 'pathname'),
     Output('login-message', 'children')],
    [Input('login-button', 'n_clicks')],
    [State('username', 'value'), State('password', 'value')]
)
def login(n_clicks, username, password):
    if n_clicks > 0:
        if username == USERNAME and password == PASSWORD:
            return '/main', ''
        else:
            return '/login', "Nom d'utilisateur ou mot de passe incorrect."
    return '/login', ''

# Callback pour définir le contenu de la page
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/main':
        return main_layout
    else:
        return login_layout

# Callback pour traiter les différentes actions sur la page principale
@app.callback(
    Output('output-container', 'children'),
    [Input('view-all-button', 'n_clicks'),
     Input('view-single-button', 'n_clicks'),
     Input('view-multiple-button', 'n_clicks'),
     Input('view-specific-button', 'n_clicks'),
     Input('upload-data', 'contents')],
    [State('activity-name', 'value'), State('activity-names', 'value'), State('upload-data', 'filename')]
)
def handle_actions(view_all, view_single, view_multiple, view_specific, file_contents, activity_name, activity_names, filename):
    ctx = dash.callback_context
    if not ctx.triggered:
        return ""
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'view-all-button' and view_all > 0:
        return html.Div("Visualisation de toutes les communautés séparées.")
    elif button_id == 'view-single-button' and view_single > 0 and activity_name:
        return html.Div(f"Visualisation de la communauté contenant l'activité : {activity_name}.")
    elif button_id == 'view-multiple-button' and view_multiple > 0 and activity_names:
        activities = [name.strip() for name in activity_names.split(',')]
        return html.Div(f"Étude des communautés pour les activités : {', '.join(activities)}.")
    elif button_id == 'view-specific-button' and view_specific > 0:
        return html.Div("Détection des communautés basées sur les ressources spécifiques (hasObjective, activityDescription, etc.).")
    elif button_id == 'upload-data' and file_contents:
        return html.Div(f"Fichier chargé : {filename}")
    else:
        return html.Div("Veuillez entrer des informations valides.", style={'color': 'red'})

# Exécuter le serveur Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


In [84]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import io
import base64

# Créer l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'Interface Dash pour Projet'
server = app.server  # Pour déploiement avec des services externes
FILE_PATH = 'projet_tpe2_1.py'

# Stocker les informations d'authentification simples (pour démonstration uniquement)
USERNAME = 'utilisateur'
PASSWORD = 'motdepasse'

# Définir le layout de l'application
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Layout de la page de connexion
login_layout = html.Div([
    html.H2("Page de Connexion"),
    dbc.Input(id='username', placeholder="Nom d'utilisateur", type='text', style={'margin-bottom': '10px'}),
    dbc.Input(id='password', placeholder='Mot de passe', type='password', style={'margin-bottom': '10px'}),
    dbc.Button('Se connecter', id='login-button', n_clicks=0, color='primary'),
    html.Div(id='login-message', style={'margin-top': '20px', 'color': 'red'})
])

# Layout de la page principale après connexion
main_layout = html.Div([
    html.H1("Interface Dash pour Étude des Communautés"),
    html.Div([
        dbc.Button("Visualiser toutes les communautés séparées", id='view-all-button', color='primary', style={'margin-bottom': '10px'}),
        dbc.Input(id='activity-name', placeholder="Entrez le nom de l'activité", type='text', style={'margin-bottom': '10px'}),
        dbc.Button("Visualiser une seule communauté", id='view-single-button', color='secondary', style={'margin-bottom': '10px'}),
        dbc.Input(id='activity-names', placeholder='Entrez deux activités ou plus, séparées par des virgules', type='text', style={'margin-bottom': '10px'}),
        dbc.Button("Étudier plusieurs activités", id='view-multiple-button', color='info', style={'margin-bottom': '10px'}),
        dbc.Button("Détecter les communautés basées sur des ressources spécifiques", id='view-specific-button', color='success', style={'margin-bottom': '10px'}),
        html.Div(id='file-upload-container', children=[
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Glissez et déposez ou ',
                    html.A('sélectionnez un fichier')
                ]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed',
                    'borderRadius': '5px', 'textAlign': 'center',
                    'margin-bottom': '10px'
                },
                multiple=False
            )
        ])
    ]),
    html.Div(id='output-container', style={'margin-top': '20px'})
])

# Callback pour la redirection après connexion
@app.callback(
    [Output('url', 'pathname'),
     Output('login-message', 'children')],
    [Input('login-button', 'n_clicks')],
    [State('username', 'value'), State('password', 'value')]
)
def login(n_clicks, username, password):
    if n_clicks > 0:
        if username == USERNAME and password == PASSWORD:
            return '/main', ''
        else:
            return '/login', "Nom d'utilisateur ou mot de passe incorrect."
    return '/login', ''

# Callback pour définir le contenu de la page
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/main':
        return main_layout
    else:
        return login_layout

# Callback pour traiter les différentes actions sur la page principale
@app.callback(
    Output('output-container', 'children'),
    [Input('view-all-button', 'n_clicks'),
     Input('view-single-button', 'n_clicks'),
     Input('view-multiple-button', 'n_clicks'),
     Input('view-specific-button', 'n_clicks'),
     Input('upload-data', 'contents')],
    [State('activity-name', 'value'), State('activity-names', 'value'), State('upload-data', 'filename')]
)
def handle_actions(view_all, view_single, view_multiple, view_specific, file_contents, activity_name, activity_names, filename):
    ctx = dash.callback_context
    if not ctx.triggered:
        return ""
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'upload-data' and file_contents:
        content_type, content_string = file_contents.split(',')
        decoded = base64.b64decode(content_string)
        try:
            # Utiliser pandas pour lire le fichier CSV
            if filename.endswith('.csv'):
                df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                return html.Div(f"Fichier chargé : {filename}")
        except Exception as e:
            return html.Div([
                'Il y a eu une erreur lors du traitement du fichier.',
                html.Br(),
                str(e)
            ])
    
    if button_id == 'view-all-button' and view_all > 0:
        return html.Div("Visualisation de toutes les communautés séparées.")
    
    elif button_id == 'view-single-button' and view_single > 0 and activity_name:
        # Créer une simple visualisation d'exemple (à remplacer par la détection de communautés)
        G = nx.complete_graph(5)
        pos = nx.spring_layout(G)
        edges = list(G.edges())
        edge_trace = go.Scatter(
            x=[],
            y=[],
            line=dict(width=0.5, color='#888'),
            hoverinfo='none',
            mode='lines'
        )
        for edge in edges:
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_trace['x'] += [x0, x1, None]
            edge_trace['y'] += [y0, y1, None]
        
        fig = go.Figure(data=[edge_trace],
                        layout=go.Layout(
                            showlegend=False,
                            hovermode='closest'
                        ))

        return dcc.Graph(figure=fig)
    
    elif button_id == 'view-multiple-button' and view_multiple > 0 and activity_names:
        activities = [name.strip() for name in activity_names.split(',')]
        return html.Div(f"Étude des communautés pour les activités : {', '.join(activities)}.")
    
    elif button_id == 'view-specific-button' and view_specific > 0:
        return html.Div("Détection des communautés basées sur les ressources spécifiques (hasObjective, activityDescription, etc.).")
    
    else:
        return html.Div("Veuillez entrer des informations valides.", style={'color': 'red'})

# Exécuter le serveur Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
